# **Stacking with QSVM and LGBM as base classifiers and Logistic regressor(LR) as meta classifier**
In this tutorial, we will see the classical stacking of QSVM,LGBM and LR algorithms, with QSVM, LGBM as base classifiers and LR as meta classifier. The stacking algorithm works as below\
1) The base classifiers are trained with training data\
2) The trained base classifiers are used to test both training data and testing data\
3) The output labels from base classifiers on training and testing data are appended as features to original training and testing data\
4) Now we train the meta classifier with appended training data and test it on appended testing data to get final prediction values



Below is detailed explanation and implementation of each of above steps


We first need to install qiskit and qiskit_machine_learning modules if not already installed. We start by importing all necessary libraries from qiskit,  qiskit\_machine\_learning and some related modules as shown in below cells.

In [ ]:
from qiskit import BasicAer
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit.library import TwoLocal
from qiskit_machine_learning.algorithms import VQC, QSVC
from qiskit_machine_learning.circuit.library import RawFeatureVector
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap, ZFeatureMap, NLocal
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit_machine_learning.kernels import QuantumKernel

In [ ]:
import sys
print(sys.path)
sys.path.append("")

['/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython']


In [ ]:
import numpy as np
from sklearn import svm, metrics
from sklearn.metrics import classification_report, confusion_matrix
from time import time
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb

###Data set
We use ethereum networks data to perform stacking. We consider 7 features for data set\
1) Degree - In degree, Out degree, Total degree\
2) Strength - In strength, Out strength, Total strength\
3) Number of neighbours

The files 'train_data.npy' and 'train_labels.npy' have 960 training data points and corresponding labels respectively, first 160 training data points are of phishing nodes and next 800 are of non-phishing nodes. The files 'test_data.npy' and 'test_labels.npy' have 11000 testing data points and corresponding labels respectively, first 1000 testing data points are of phishing nodes and next 10000 are of non-phishing nodes.


let us choose 320 training data points to perform stacking, 160 phishing and 160 non-phishing. Even though the real life data is highly imbalanced, we choose equal number of phishing and non-phishing nodes for training our classifiers and then test on imbalanced testing data. With different experiments on this data set, taking balanced training data proved more effective in giving good results

In [ ]:
train_data = np.load('train_data.npy')
train_labels = np.load('train_labels.npy')
test_data = np.load('test_data.npy')
test_labels = np.load('test_labels.npy')
train_data = train_data[:320]
train_labels = train_labels[:320]

# Classical SVM training and testing
We perform classical support vector machine classification to compare our final results

In [ ]:
#training classical SVM with rbf kernel

print("*** Training a classical SVM classifier with rbf Kernel ***")

#converting two dimensional labels to 1D
train_labels_svm = train_labels[:,0]
test_labels_svm = test_labels[:,0]

clf = svm.SVC(kernel='rbf')
start_time = time()
clf.fit(train_data, train_labels_svm)
end_time = time()
duration = end_time - start_time
print("training time for classical SVM : ", duration)
y_pred=clf.predict(test_data)
print(confusion_matrix(test_labels_svm, y_pred))
print(classification_report(test_labels_svm, y_pred))

*** Training a classical SVM classifier with rbf Kernel ***
training time for classical SVM :  0.009143829345703125
[[9676  324]
 [ 559  441]]
              precision    recall  f1-score   support

           0       0.95      0.97      0.96     10000
           1       0.58      0.44      0.50      1000

    accuracy                           0.92     11000
   macro avg       0.76      0.70      0.73     11000
weighted avg       0.91      0.92      0.91     11000



# **Stacking**
###Initializing the classifiers
Initialize the two base classifiers QSVM and LGBM. ZZ feature map is used to calculate kernel matrix for QSVM classifier. 
It can be calculated using below equation\
$K(\vec{x_i},\vec{x_j}) =K_{ij}= \vert\langle\phi^\dagger(\vec{x_j})\vert\phi(\vec{x_i})\rangle\vert^2$, where $x_i,x_j\in X$(training data set) and $\phi$ represents feature map 

Statevector simulator is used to simulate the results of quantum computer, it can be replaced with backend for harware results

In this tutorial we are considering two base classifiers, the second one is classical ML algorithm LGBM

Over these two base classifiers we use Logistic regression as meta classifier

In [ ]:
seed = 1376

#feature dimensions
feature_dim = train_data.shape[1]

#feature map to calculate kernel of QSVM
feature_map = ZZFeatureMap(feature_dim)

#initialize LGBM classifier
clf = lgb.LGBMClassifier()

#initialize kernel of QSVM
kernel = QuantumKernel(feature_map=feature_map,
                             quantum_instance=QuantumInstance(BasicAer.get_backend('statevector_simulator'),
                                            shots=1,
                                            seed_simulator=seed,
                                                              seed_transpiler=seed))

#initialize QSVM
qsvc = QSVC(quantum_kernel=kernel)
lr = LogisticRegression()

###Training the base classifiers
In the following cells we train the base classifiers QSVM and LGBM using 320 training data points, then we predict the labels of both training data and testing data using the trained classifiers, the labels we obtained are added as features to initial training and testing data sets, thus our final training data and testing data with appended features consists of total 9 features(one label is added from each classifier)

In [ ]:
#train base classifiers and append features to data

def level_0():
    
    #Use QSVM and LGBM on train data, append the predicted labels on train data to train data features
    clf.fit(train_data,train_labels_svm)
    b = clf.predict(train_data)
    label_2 = np.reshape(b,(len(b),1))
    qsvc.fit(train_data, train_labels_svm)
    c = qsvc.predict(train_data)
    label_3 = np.reshape(c,(len(c),1))
    t1 = np.append(train_data,label_2,1)
    train_added = np.append(t1,label_3,1)
    
    #append the predicted labels on test data to test data features
    e = clf.predict(test_data)
    label_5 = np.reshape(e,(len(e),1))
    f = qsvc.predict(test_data)
    label_6 = np.reshape(f,(len(f),1))
    t3 = np.append(test_data,label_5,1)
    test_added = np.append(t3,label_6,1)
    
    return train_added,test_added

In [ ]:
# initialize logistic regressor and get appended train and test data

start_time = time()
train_added, test_added = level_0()
end_time = time()
duration = end_time - start_time
print("training time : ", duration)

training time :  193.53476977348328


###Training the meta classifier
The LR classifier is now trained with training data of 320 data points with 9 features (7 features + one feature from prediction label of that particular training data point using QSVM + one feature from prediction label of that particular training data point using LGBM)

In [ ]:
#train logistic regressor using appended train data

start_time = time()
print("training_data", train_data.shape)
lr.fit(train_added, train_labels_svm)
end_time = time()
duration = end_time - start_time
print("training time : ", duration)

training_data (320, 7)
training time :  0.03599047660827637


###Get final prediction
We test the appended data set with 9 features (7 features + one feature from prediction label of that particular test data point using QSVM + one feature from prediction label of that particular test data point using LGBM) using the trained meta classifier (LR) and get our final prediction results

In [ ]:
# predict on appended test data

start_time1 = time()
y_pred_1 = lr.predict(test_added)
end_time1 = time()
duration1 = end_time1 - start_time1
print("testing time : ", duration1)
print(classification_report(test_labels_svm,y_pred_1))
print(confusion_matrix(test_labels_svm,y_pred_1))

testing time :  0.001850128173828125
              precision    recall  f1-score   support

           0       1.00      0.91      0.95     10000
           1       0.53      0.97      0.68      1000

    accuracy                           0.92     11000
   macro avg       0.76      0.94      0.82     11000
weighted avg       0.95      0.92      0.93     11000

[[9130  870]
 [  33  967]]
